In [26]:
using MeshCat
using RigidBodyDynamics
using MeshCatMechanisms

In [35]:
vis = Visualizer()
#IJuliaCell(vis)
open(vis)

┌ Info: Serving MeshCat visualizer at http://127.0.0.1:8702
└ @ MeshCat /Users/roberto/.julia/packages/MeshCat/GlxLQ/src/servers.jl:7
┌ Info: Listening on: Sockets.InetAddr{Sockets.IPv4}(ip"127.0.0.1", 0x21fe)
└ @ HTTP.Servers /Users/roberto/.julia/packages/HTTP/YjRCz/src/Servers.jl:301


Process(`open http://127.0.0.1:8702`, ProcessExited(0))

┌ Info: Accept (0):  🔗    0↑     0↓    0s 127.0.0.1:8702:8702 ≣16
└ @ HTTP.Servers /Users/roberto/.julia/packages/HTTP/YjRCz/src/Servers.jl:343
┌ Info: Accept (1):  🔗    0↑     0↓    0s 127.0.0.1:8702:8702 ≣16
└ @ HTTP.Servers /Users/roberto/.julia/packages/HTTP/YjRCz/src/Servers.jl:343
┌ Info: Accept (2):  🔗    0↑     0↓    0s 127.0.0.1:8702:8702 ≣16
└ @ HTTP.Servers /Users/roberto/.julia/packages/HTTP/YjRCz/src/Servers.jl:343


In [36]:
urdf = joinpath(dirname(pathof(MeshCatMechanisms)), "..", "test", "urdf", "Acrobot.urdf")
robot = parse_urdf(Float64, urdf)
delete!(vis)
mvis = MechanismVisualizer(robot, URDFVisuals(urdf), vis)
set_configuration!(mvis, [0.0, 0.0])

┌ Info: Closed (0):  💀    2↑     2↓🔒 116s 127.0.0.1:8702:8702 ≣16
└ @ HTTP.Servers /Users/roberto/.julia/packages/HTTP/YjRCz/src/Servers.jl:351
┌ Info: Closed (1):  💀    5↑     5↓🔒 115s 127.0.0.1:8702:8702 ≣16
└ @ HTTP.Servers /Users/roberto/.julia/packages/HTTP/YjRCz/src/Servers.jl:351


In [33]:
state = MechanismState(robot, randn(2), randn(2))
t, q, v = simulate(state, 20.0);

In [34]:
setanimation!(mvis, t, q)

┌ Info: Closed (0):  💀    2↑     2↓🔒 116s 127.0.0.1:8701:8701 ≣16
└ @ HTTP.Servers /Users/roberto/.julia/packages/HTTP/YjRCz/src/Servers.jl:351
┌ Info: Closed (1):  💀    5↑     5↓🔒 116s 127.0.0.1:8701:8701 ≣16
└ @ HTTP.Servers /Users/roberto/.julia/packages/HTTP/YjRCz/src/Servers.jl:351
Error handling websocket connection:
WebSockets.WebSocketClosedError("ws|server respond to OPCODE_CLOSE 1001:Going Away")┌ Info: Closed (2):  💀    1↑     1↓🔒 130s 127.0.0.1:8701:8701 ≣16
└ @ HTTP.Servers /Users/roberto/.julia/packages/HTTP/YjRCz/src/Servers.jl:351


In [23]:
?simulate

search: simulate



```julia
simulate(state0, final_time)
simulate(state0, final_time, control!; Δt, stabilization_gains)

```

Basic `Mechanism` simulation: integrate the state from time $0$ to `final_time` starting from the initial state `state0`. Return a `Vector` of times, as well as `Vector`s of configuration vectors and velocity vectors at these times.

Optionally, a function (or other callable) can be passed in as the third argument (`control!`). `control!` will be called at each time step of the simulation and allows you to specify joint torques given the time and the state of the `Mechanism`. It should look like this:

```julia
function control!(torques::AbstractVector, t, state::MechanismState)
    rand!(torques) # for example
end
```

The integration time step can be specified using the `Δt` keyword argument (defaults to `1e-4`).

The `stabilization_gains` keyword argument can be used to set PD gains for Baumgarte stabilization, which can be used to prevent separation of non-tree (loop) joints. See Featherstone (2008), section 8.3 for more information. There are several options for specifying gains:

  * `nothing` can be used to completely disable Baumgarte stabilization.
  * Gains can be specifed on a per-joint basis using any `AbstractDict{JointID, <:RigidBodyDynamics.PDControl.SE3PDGains}`, which maps the `JointID` for the non-tree joints of the mechanism to the gains for that joint.
  * As a special case of the second option, the same gains can be used for all joints by passing in a `RigidBodyDynamics.CustomCollections.ConstDict{JointID}`.

The [`default_constraint_stabilization_gains`](@ref) function is called to produce the default gains, which use the last option.

Uses `MuntheKaasIntegrator`. See [`RigidBodyDynamics.OdeIntegrators.MuntheKaasIntegrator`](@ref) for a lower level interface with more options.
